In [1]:
#!pip install graphviz

import dspy
import os
import openai
import json
from dotenv import load_dotenv
from IPython.display import display

In [2]:
def set_dspy():
    # ==============set openAI enviroment=========
    # Path to your API key file
    key_file_path = "openai_api_key.txt"

    # Load the API key from the file
    with open(key_file_path, "r") as file:
        openai_api_key = file.read().strip()

    # Set the API key as an environment variable
    os.environ["OPENAI_API_KEY"] = openai_api_key
    openai.api_key = os.environ["OPENAI_API_KEY"]
    turbo = dspy.OpenAI(model="gpt-3.5-turbo", max_tokens=2000, temperature=0.5)
    dspy.settings.configure(lm=turbo)
    return turbo
    # ==============end of set openAI enviroment=========


def set_dspy_hardcode_openai_key():
    os.environ["OPENAI_API_KEY"] = (
        "sk-proj-yourapikeyhere"
    )
    openai.api_key = os.environ["OPENAI_API_KEY"]
    turbo = dspy.OpenAI(model="gpt-3.5-turbo",  temperature=0, max_tokens=2000)
    dspy.settings.configure(lm=turbo)
    return turbo

turbo=set_dspy()
# comment out set_dspy() and use set_dspy_hardcode_openai_key is your option
# turbo=set_dspy_hardcode_openai_key()

In [3]:
def load_text_file(file_path):
    """
    Load a text file and return its contents as a string.

    Parameters:
    file_path (str): The path to the text file.

    Returns:
    str: The contents of the text file.
    """
    try:
        with open(file_path, "r") as file:
            contents = file.read()
        return contents
    except FileNotFoundError:
        return "File not found."
    except Exception as e:
        return f"An error occurred: {e}"


In [4]:
conversation=load_text_file("conversation.txt")
print(conversation)

Alice: Hey Bob, I just got a strange email from support@banksecure.com. It says I need to verify my account details urgently. The subject line was "Urgent: Verify Your Account Now". The email looks suspicious to me.

Bob: Hi Alice, that does sound fishy. Can you forward me the email? I’ll take a look at the headers to see where it came from.

Alice: Sure, forwarding it now.

Bob: Got it. Let’s see... The email came from IP address 192.168.10.45, but the domain banksecure.com is not their official domain. It's actually registered to someone in Russia.

Alice: That’s definitely not right. Should I be worried?

Bob: We should investigate further. Did you click on any links or download any attachments?

Alice: I did click on a link that took me to a page asking for my login credentials. I didn't enter anything though. The URL was http://banksecure-verification.com/login.

Bob: Good call on not entering your details. Let’s check the URL. This domain was just registered two days ago. It’s hi

In [5]:
class SITXGenerator(dspy.Signature):
    """Describe a conversation in STIX, which stands for Structured Threat Information eXpression, is a standardized language for representing cyber threat information."""

    question: str = dspy.InputField(
        desc="a conversation describing a cyber incident between an IT Security Specialist and an employee."
    )

    answer: str = dspy.OutputField(
        desc="the formalized STIX in JSON representing cyber threat information based on the conversation, e.g., [{object 1}, {object 2}, ... {object n}]"
    )

In [6]:
def generate_answer_CoT(signature, conversation, output_file):
    generate_answer = dspy.ChainOfThought(signature)
    answer = generate_answer(question=conversation).answer  # here we use the module

    with open(output_file, "w") as json_file:
        result = json.loads(answer)
        print(answer)
        json.dump(result, json_file, indent=4)
    print(f"The evidence has been saved to the file {output_file}")

In [7]:
output_file = "03_output.json"
generate_answer_CoT(
    SITXGenerator,
    conversation,
    output_file,
)

[
  {
    "email": {
      "sender": "support@banksecure.com",
      "subject": "Urgent: Verify Your Account Now",
      "headers": {
        "IP_address": "192.168.10.45",
        "domain": "banksecure.com",
        "registered_to": "Russia"
      },
      "links_clicked": [
        {
          "URL": "http://banksecure-verification.com/login",
          "timestamp": "10:15 AM"
        },
        {
          "URL": "http://banksecure-verification.com/account-details",
          "timestamp": "10:17 AM"
        }
      ],
      "attachments": [
        {
          "file_name": "AccountDetails.exe",
          "created_at": "10:20 AM",
          "MD5_hash": "e99a18c428cb38d5f260853678922e03",
          "status": "known_malware"
        }
      ]
    }
  },
  {
    "actions_taken": [
      "Clear browser history and cache",
      "Run full antivirus scan",
      "Provide browser history entries and cookies",
      "Quarantine suspicious file",
      "Check network connections",
      "Rese

In [8]:
turbo.inspect_history(n=1)




Describe a conversation in STIX, which stands for Structured Threat Information eXpression, is a standardized language for representing cyber threat information.

---

Follow the following format.

Question: a conversation describing a cyber incident between an IT Security Specialist and an employee.
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: the formalized STIX in JSON representing cyber threat information based on the conversation, e.g., [{object 1}, {object 2}, ... {object n}]

---

Question: Alice: Hey Bob, I just got a strange email from support@banksecure.com. It says I need to verify my account details urgently. The subject line was "Urgent: Verify Your Account Now". The email looks suspicious to me. Bob: Hi Alice, that does sound fishy. Can you forward me the email? I’ll take a look at the headers to see where it came from. Alice: Sure, forwarding it now. Bob: Got it. Let’s see... The email came from IP address 192.168.10.45, but th

'\n\n\nDescribe a conversation in STIX, which stands for Structured Threat Information eXpression, is a standardized language for representing cyber threat information.\n\n---\n\nFollow the following format.\n\nQuestion: a conversation describing a cyber incident between an IT Security Specialist and an employee.\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\nAnswer: the formalized STIX in JSON representing cyber threat information based on the conversation, e.g., [{object 1}, {object 2}, ... {object n}]\n\n---\n\nQuestion: Alice: Hey Bob, I just got a strange email from support@banksecure.com. It says I need to verify my account details urgently. The subject line was "Urgent: Verify Your Account Now". The email looks suspicious to me. Bob: Hi Alice, that does sound fishy. Can you forward me the email? I’ll take a look at the headers to see where it came from. Alice: Sure, forwarding it now. Bob: Got it. Let’s see... The email came from IP address 192.